## Maximum Likelyhood for PGM

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Class definitions - RV, Distribution and Category

In [2]:
class RV:
    def __init__(self, cattup,name):
        self.name = name
        self.categories = cattup
        self.catcountdict = {}
        for category in self.categories:
            self.catcountdict[category] = Category(category)
    def __str__(self):
        return_string = " \t < RV: " + self.name + " > \n"
        count = 0
        for category in self.catcountdict:
            return_string = return_string + category+ "\t\t"+ str(self.catcountdict[category].count) + "\n"
            count = count + self.catcountdict[category].count
        return_string = return_string + "total count: " + str(count)
        return return_string
    def clear_count(self):
        for cat in self.catcountdict:
            self.catcountdict[cat].count = 0
        
class Distribution:
    def __init__(self,randomvar):
        self.name = randomvar.name
        categories = randomvar.categories
        catcountdict = randomvar.catcountdict
        distdict = {}
        total = 0
        for category in categories:
            total = total + catcountdict[category].count
        for category in categories:
            distdict[category] = catcountdict[category].count/(1.0*total)
        
        self.categories = categories
        self.distdict = distdict
        self.total = total
    def __str__(self):
        return_string = " \t < Distribution: " + self.name + " > \n"
        for category in self.distdict:
            return_string = return_string + category+ "\t\t"+ str(self.distdict[category]) + "\n"
            
        return return_string
    def to_csv(self):
        table = []
        for category in self.distdict:
            table.append([category,self.distdict[category]])
        df = pd.DataFrame(data=table, columns=['category','prob'])
        name = "prior_" + self.name + ".csv"
        df.to_csv(name,sep=',')
    def to_emdw_csv(self):
        table = []
        ind = 0
        for category in self.distdict:
            table.append([ind,self.distdict[category]])
            ind = ind +1
        df = pd.DataFrame(data=table)
        name = "prior_" + self.name + ".csv"
        df.to_csv(name,sep=',',index=False,header=False)

class Category:
    def __init__(self,category,catcount=0):
        self.name = category
        self.count = catcount
    def increment(self):
        self.count = self.count + 1
        



### Read in the graph

In [3]:
x = pd.read_csv('car.csv')


### Store the possible categories per RV

In [4]:
# find out how many categories each RV can take
rvcategorydict = {}
for rv in x.columns:
    rvcategorydict[rv] = tuple(set([item[0] for item in x[[rv]].values]))
for rv in rvcategorydict:
    print(rv,rvcategorydict[rv])

doors ('5more', '3', '2', '4')
safety ('high', 'low', 'med')
persons ('more', '2', '4')
maint ('vhigh', 'high', 'low', 'med')
verdict ('vgood', 'acc', 'unacc', 'good')
buying ('vhigh', 'high', 'low', 'med')
lug_boot ('small', 'big', 'med')


### Set up the RV dictionary

In [5]:
rvdictionary = {}

for rv in rvcategorydict:
    rvdictionary[rv] = RV(rvcategorydict[rv],name=rv)
    

## Calculate the prior probabilities

In [6]:
for rv in x.columns:
    for item in x[[rv]].values:
        rvdictionary[rv].catcountdict[item[0]].increment()

distributiondict = {}

for rv in rvdictionary:
    distributiondict[rv] = Distribution(rvdictionary[rv])

for rv in rvdictionary:
    print(rvdictionary[rv])
    
for rv in distributiondict:
    print(distributiondict[rv])
    Distribution.to_emdw_csv(distributiondict[rv])

 	 < RV: buying > 
vhigh		432
high		432
low		432
med		432
total count: 1728
 	 < RV: safety > 
high		576
low		576
med		576
total count: 1728
 	 < RV: persons > 
more		576
2		576
4		576
total count: 1728
 	 < RV: maint > 
vhigh		432
high		432
low		432
med		432
total count: 1728
 	 < RV: verdict > 
vgood		65
acc		384
unacc		1210
good		69
total count: 1728
 	 < RV: doors > 
5more		432
3		432
2		432
4		432
total count: 1728
 	 < RV: lug_boot > 
small		576
big		576
med		576
total count: 1728
 	 < Distribution: doors > 
5more		0.25
3		0.25
2		0.25
4		0.25

 	 < Distribution: safety > 
high		0.3333333333333333
low		0.3333333333333333
med		0.3333333333333333

 	 < Distribution: persons > 
more		0.3333333333333333
2		0.3333333333333333
4		0.3333333333333333

 	 < Distribution: maint > 
vhigh		0.25
high		0.25
low		0.25
med		0.25

 	 < Distribution: verdict > 
vgood		0.03761574074074074
acc		0.2222222222222222
unacc		0.7002314814814815
good		0.03993055555555555

 	 < Distribution: buying > 
vhigh

## Maximum likelyhood

### Define the graph structure

In [7]:
# family = {'C':['A','B'], 'D':['C'],'F':['D','B','C']}
family = {'verdict':['buying','maint', 'doors', 'persons', 'lug_boot', 'safety']}

In [8]:
for child in family:
    print("child: ",child,"\nparents: ", family[child])

child:  verdict 
parents:  ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']


### Compute the maximum likelyhood counts

In [9]:
class ChildContainer:
    def __init__(self,rvobj, childcategorydict,childrvdict):
        self.rvobj = rvobj
        self.catdict = childcategorydict

In [26]:


# dictionaries that will contain the data to calculate MLE for the specific graph structure   
childrvdict = {}
childtotals = {}
graphdict = {}
# print(childrvdict,childcategorydict)

for rv, item in rvdictionary.items():
    item.clear_count()

for child, parents in family.items():
    childcategorydict = {}
    print(child)
    dtlist = []
    partuplist = []
    
    for item in x[[*parents]].values:
        partuplist.append((tuple(parents),tuple(item)))
    childtotals[child] = partuplist
   
    for category in rvcategorydict[child]:
        partuplistcat = []
        print(category)
        childcategorydict[category] = {}

        for (i,item) in enumerate(x[[*parents]].values):
            if x[[child]].values[i][0] == category:
                partuplistcat.append((tuple(parents),tuple(item)))
        childcategorydict[child] = partuplistcat

    graphdict[child] = childcategorydict
    
childsetdict = {}
for child in graphdict:
    childsetdict[child] = set(graphdict[child])
    print(childsetdict[child])
#     
#     print(childcategorydict['unacc']['buying'])
               
    

    #     for category in childcategorydict:
    #         for rv in childcategorydict[category]:
    #             print("thisone:,",rv, childcategorydict[category][rv])

#     for rv, rvobj in rvdictionary.items():
#         if rv != child:
#             childcategorydict[category][rv] = rvobj

#         print(childcategorydict['unacc']['buying'])
#         for rv in parents:
#             for category in childcategorydict:
#                 for (i,item) in enumerate(x[[rv]].values):
#     #                 print(x[[child]].values[i][0],category)
#                     if x[[child]].values[i][0] == category:
#                         print("yes")
    #                     print("cat:-------",category)
    #                     print(rv)
    #                     print("this:",childcategorydict[category])
    #                     childcategorydict[category][rv].catcountdict[x[[rv]].values[i]].increment()
#     graphdict[child] = childcategorydict
#     graphdict[child] = childcontainer(rvdictionary[child],childcategorydict )
                                                    



verdict
vgood
acc
unacc
good
{'vgood', 'acc', 'verdict', 'unacc', 'good'}


In [36]:
for child in childsetdict:
    print(childsetdict[child])

childsetdictcat = {}

    
for child, categoryobj in graphdict.items():
    catsets = {}
    for category in categoryobj:
        catsets[category] = set(graphdict[child][category])
    childsetdictcat[child] = catsets
    

        


{'vgood', 'acc', 'verdict', 'unacc', 'good'}


In [20]:
for child,parents in family.items():
    print("child:",child)
    for category in graphdict[child]:
        print("-------------------------------category:---------------",category)
        for rv in parents:
#         print("category: ",category)
#         tempdict = graphdict[child] 
# #         print(category, tempdict[category] )
            print(tempdict[category][rv])


child: verdict


In [71]:
# dictionaries that will contain the data to calculate MLE for the specific graph structure   
childict = {}
childict_true = {}

for child, parents in family:
    print(child,parents)
    
#     # list of all the parents for a specific child node
#     parlist = [parent for parent in family[child]]
#     # lists containing entries for all parent combinations as well as ones only for when the child is True
#     partuplist = []
#     partuplist_true = []
#     # iterate through all entries and store the data in the lists
    
#     for (check,item) in zip(checklist, x[[*parlist]].values):
        
#     checklist = []
#     for item in x[child].values:
#         checklist.append(item)
#     for (check,item) in zip(checklist, x[[*parlist]].values):
#         partuplist.append((tuple(parlist),tuple(item)))
#         if check == 'vgood' or 'acc':
#             partuplist_true.append((tuple(parlist),tuple(item)))
    
#     # add the lists to the dictionaries
#     childict[child] =partuplist
#     childict_true[child] =partuplist_true
    
# get the set of all the parent combinations for each child (all other combinations will have probability 0 or Ebs)
childsetdict = {}
for child in childict:
    childsetdict[child] = set(childict[child])
# for child in childsetdict:
#     print("child: ",child,", Parents: ", childsetdict[child])
    
# dictionaries to hold the prelim counts
countdict = {}
countdict_true = {}
# initializing counts to 0
for child in childsetdict:
    for parents in childsetdict[child]:
        countdict[parents] = (0, child)
        countdict_true[parents] = (0, child)
# sum all occurances of each parent combination for each child   
for child in childict:
    for entry in childict[child]:
        countdict[entry] = (countdict[entry][0] + 1,child)
# sum all occurances of each parent combination for each child when the child is True
for child in childict_true:
    for entry in childict_true[child]:
        countdict_true[entry] = (countdict_true[entry][0] + 1,child)
# calculate MLE from the two counts above
mledict = {}       
for entry in countdict:
    mledict[entry] = (countdict_true[entry][0]/countdict[entry][0],countdict_true[entry][1])


ValueError: too many values to unpack (expected 2)

In [15]:
class DT:
    def __init__(self, catlist,vallist):
        self.list =[]
        for (i, cat) in enumerate(catlist):
            self.list.append(DT_item(cat,vallist[i]))
    def __str__(self):
        return_string = "assignments: "
        for item in self.list:
            return_string =  return_string + item.cat + " = " + item.val + ", "
        return return_string
                
class DT_item:
    def __init__(self,cat,val):
        self.cat = cat
        self.val = val

In [16]:
# dictionaries that will contain the data to calculate MLE for the specific graph structure   
childict = {}
childict_true = {}

for child in family:
    # list of all the parents for a specific child node
    parlist = [parent for parent in family[child]]
    # lists containing entries for all parent combinations as well as ones only for when the child is True
    partuplist = []
    partuplist_true = []
    # iterate through all entries and store the data in the lists
    checklist = []
    dtlist = []
    for item in x[child].values:
        checklist.append(item)
    for (check,item) in zip(checklist, x[[*parlist]].values):
        dtlist.append(DT(parlist,item))
        partuplist.append((tuple(parlist),tuple(item)))
        if check == 'vgood' or 'acc':
            partuplist_true.append((tuple(parlist),tuple(item)))
#             print(tuple(parlist),tuple(item))
    
    # add the lists to the dictionaries
    childict[child] =partuplist
    childict_true[child] =partuplist_true
    for item in dtlist:
        print(item)
    
# get the set of all the parent combinations for each child (all other combinations will have probability 0 or Ebs)
childsetdict = {}
for child in childict:
    childsetdict[child] = set(childict[child])
# for child in childsetdict:
#     print("child: ",child,", Parents: ", childsetdict[child])
    
# dictionaries to hold the prelim counts
countdict = {}
countdict_true = {}
# initializing counts to 0
for child in childsetdict:
    for parents in childsetdict[child]:
        countdict[parents] = (0, child)
        countdict_true[parents] = (0, child)
# sum all occurances of each parent combination for each child   
for child in childict:
    for entry in childict[child]:
        countdict[entry] = (countdict[entry][0] + 1,child)
# sum all occurances of each parent combination for each child when the child is True
for child in childict_true:
    for entry in childict_true[child]:
        countdict_true[entry] = (countdict_true[entry][0] + 1,child)
# calculate MLE from the two counts above
mledict = {}       
for entry in countdict:
    mledict[entry] = (countdict_true[entry][0]/countdict[entry][0],countdict_true[entry][1])


assignments: buying = vhigh, maint = vhigh, doors = 2, persons = 2, lug_boot = small, safety = low, 
assignments: buying = vhigh, maint = vhigh, doors = 2, persons = 2, lug_boot = small, safety = med, 
assignments: buying = vhigh, maint = vhigh, doors = 2, persons = 2, lug_boot = small, safety = high, 
assignments: buying = vhigh, maint = vhigh, doors = 2, persons = 2, lug_boot = med, safety = low, 
assignments: buying = vhigh, maint = vhigh, doors = 2, persons = 2, lug_boot = med, safety = med, 
assignments: buying = vhigh, maint = vhigh, doors = 2, persons = 2, lug_boot = med, safety = high, 
assignments: buying = vhigh, maint = vhigh, doors = 2, persons = 2, lug_boot = big, safety = low, 
assignments: buying = vhigh, maint = vhigh, doors = 2, persons = 2, lug_boot = big, safety = med, 
assignments: buying = vhigh, maint = vhigh, doors = 2, persons = 2, lug_boot = big, safety = high, 
assignments: buying = vhigh, maint = vhigh, doors = 2, persons = 4, lug_boot = small, safety = low, 

### Print the MLE conditional probabilities for each child

In [ ]:
# print the MLE conditional probabilities for each child
# table = np.empty([len(mledict)*2,len(x.columns)+1],dtype=str)
table = []

# i = 0
# for entry in mledict:
#     j = 0
#     for val in entry[1]:
#         table[i][j] = val 
#         print(table[i][j])
#         j = j + 1
#     table[i][j] =  1
#     table[i][1+j] =  mledict[entry][0]
#     i = i + 1
#     table[i][j] = 0
#     table[i][j+1] = 1 -  mledict[entry][0]
#     i = i + 1
    
for entry in mledict:
    row = []
    for val in entry[1]:
        row.append(val)
    row.append(val)
#     print(mledict[entry][0])
    row.append(mledict[entry][0])
    table.append(row)
    
# print(table[0:100,:])
#     print("P(",mledict[entry][1]," = 1 |",entry,") = ", mledict[entry][0])
#     print("P(",mledict[entry][1]," = 0 |",entry,") = ", 1-mledict[entry][0])
    

In [ ]:
df = pd.DataFrame(data=table, columns=[c for c in x.columns] + ['condprob'])
df.to_csv('out.csv',sep=',')

In [ ]:
# print(table[1000:1100,:])